In [1]:
import torch

In [ ]:
x = torch.tensor([[2.0, 2.0, 2.0], [3.0,3.0,3.0], [4.0,4.0,4.0],[5.0, 6.0, 7.0]], requires_grad=True)# giá trị đưa vào phải là float
# requires_grad=True cho phép tính đạo hàm
y = x ** 2 
y = y.backward(torch.ones_like(x)) # truyền vào một tensor cùng kích thước với x để tính đạo hàm 
# y là một scalar nên phải truyền vào một tensor cùng kích thước với x
# nếu y là một tensor có kích thước khác thì sẽ báo lỗi
# với x là một số x = torch.tensor(2.0, requires_grad = True) , y.backward() sẽ không cần truyền vào một tensor cùng kích thước với x
# y.backward() sẽ tính đạo hàm của y theo x và lưu vào x.grad
print(x.grad)  


tensor([[ 4.,  4.,  4.],
        [ 6.,  6.,  6.],
        [ 8.,  8.,  8.],
        [10., 12., 14.]])


Khi đặt requires_grad=True, PyTorch bắt đầu theo dõi toàn bộ phép toán của tensor đó.
Mỗi phép toán sẽ được ghi lại trong một đồ thị ngược (backward graph).
Khi gọi y.backward(), PyTorch sẽ lan truyền ngược (backpropagation) và tính đạo hàm.

.backward() mặc định tính đạo hàm của một giá trị đầu ra duy nhất (thường là loss trong mạng nơ-ron), và truyền gradient ngược qua từng phép toán đã thực hiện.

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

# z = w*x + b
z = w * x + b
z.backward()

print(x.grad)  
print(w.grad)  
print(b.grad)  


tensor(2.)
tensor(1.)
tensor(1.)


## Xóa gradient cũ

Trong PyTorch, khi gọi .backward() nhiều lần, các giá trị gradient sẽ bị cộng dồn (accumulate) thay vì được ghi đè.

In [19]:
# x.grad.zero_()  # Đặt lại đạo hàm về 0
x = torch.tensor(4.0, requires_grad=True)
z = w * x**2 + b
z.backward()
print(x.grad)  # In ra đạo hàm mới của x
z = w * x**3 + b
z.backward()
print(x.grad)  # In ra đạo hàm mới của x, sẽ cộng dồn với đạo hàm trước đó
z = w * x
z.backward()
print(x.grad)  # In ra đạo hàm mới của x, sẽ cộng dồn với đạo hàm trước đó

tensor(16.)
tensor(112.)
tensor(114.)


Để tránh cộng dồn ta xóa gradient cũ

In [20]:
x.grad.zero_()  # Đặt lại đạo hàm về 0
x = torch.tensor(4.0, requires_grad=True)
z = w * x**2 + b
z.backward()
print(x.grad)  # In ra đạo hàm mới của x

x.grad.zero_()  # Đặt lại đạo hàm về 0
z = w * x**3 + b
z.backward()
print(x.grad)  # In ra đạo hàm mới của x, sẽ cộng dồn với đạo hàm trước đó

x.grad.zero_()  # Đặt lại đạo hàm về 0
z = w * x
z.backward()
print(x.grad)  # In ra đạo hàm mới của x, sẽ cộng dồn với đạo hàm trước đó

tensor(16.)
tensor(96.)
tensor(2.)


### Retain_graph

Có thể sử dụng phương thức `retain_graph=True` trong hàm `backward()`. Điều này cho phép tính đạo hàm nhiều lần mà không cần phải đặt lại đồ thị tính toán. Dưới đây là ví dụ:

Khi không dùng retain_graph = True

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
y.backward()
print(x.grad)  # In ra đạo hàm của y theo x

y.backward() 
# gọi lại backward sẽ báo lỗi vì x.grad đã được tính và không thể tính đạo hàm lại


tensor(4.)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

Khi gọi retain_graph = True

In [28]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2

y.backward(retain_graph=True)  # OK
print("grad lần 1:", x.grad)   # 8.0

x.grad.zero_()
y.backward()  # OK, vì retain_graph ở trên
print("grad lần 2:", x.grad)  # 8.0

grad lần 1: tensor(4.)
grad lần 2: tensor(4.)


Tính đạo hàm bậc 2

In [ ]:
x = torch.tensor(3.0, requires_grad=True)

y = x ** 3  
# đạo hàm cấp 1
dy_dx = torch.autograd.grad(y, x, create_graph=True)[0] 

# đạo hàm cấp 2
d2y_dx2 = torch.autograd.grad(dy_dx, x)[0]  

print("dy/dx:", dy_dx.item())      
print("d2y/dx2:", d2y_dx2.item())  


dy/dx: 27.0
d2y/dx2: 18.0


| Tính năng           | `retain_graph=True`                             | `x.grad.zero_()`                          |
| ------------------- | ----------------------------------------------- | ----------------------------------------- |
| **Liên quan đến**   | Đồ thị tính toán (computation graph)            | Gradient lưu trong `x.grad`               |
| **Mục đích chính**  | Cho phép `.backward()` chạy **lại**             | Reset gradient về **0**                   |
| **Dùng khi nào?**   | Cần tính đạo hàm nhiều lần **trên cùng đồ thị** | Trước mỗi batch hoặc mỗi lần `backward()` |
| **Nếu không dùng?** | `.backward()` lần 2 sẽ báo lỗi                  | Gradient sẽ bị **cộng dồn**               |


| Thuộc tính                  | `.backward()`              | `torch.autograd.grad()`             |
| --------------------------- | -------------------------- | ----------------------------------- |
| Trả về kết quả?             | ❌ Không (cập nhật `.grad`) | ✅ Có (trả về gradient)              |
| Có ghi vào `.grad`?         | ✅ Có                       | ❌ Không                             |
| Dùng trong đạo hàm bậc 2?   | ❌ Không tốt lắm            | ✅ Có thể tạo graph để đạo hàm bậc 2 |
| Dễ điều khiển hơn cho grad? | ❌ Ít linh hoạt             | ✅ Linh hoạt và chính xác hơn        |


Khi nào dùng autograd
| Trường hợp                                         | Dùng cái nào?                      |
| -------------------------------------------------- | ---------------------------------- |
| Huấn luyện bình thường                             | `.backward()`                      |
| Muốn lấy đạo hàm **mà không làm bẩn .grad**        | `autograd.grad()`                  |
| Tính đạo hàm bậc hai (gradient của gradient)       | `autograd.grad(create_graph=True)` |
| Tính grad theo cách **tùy biến (vector-Jacobian)** | `autograd.grad()`                  |


# Loss func cơ bản

In [34]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, dtype=torch.float32)
loss = (x - y) ** 2
loss.backward()  # Tính đạo hàm của loss theo x
print("Loss:", loss.item())
print("Gradient of x:", x.grad.item())  # In ra đạo hàm của loss theo x


Loss: 1.0
Gradient of x: -2.0
